In [ ]:
import os
import tensorflow as tf
from dotenv import load_dotenv
from usd_idr_forecasting.trainers import Retrainer
from usd_idr_forecasting.configs import ProjectConfig
import pandas as pd
load_dotenv()
PROJECT_WORKING_DIR = os.getenv("PROJECT_WORKING_DIR")

In [ ]:
config_path = os.path.join(PROJECT_WORKING_DIR, 'project_configs.yaml')
project_config = ProjectConfig.from_yaml(config_path)
project_config

In [ ]:
lstm_retrainer = Retrainer(
	config=project_config, 
	batch_sizes=[16, 32],
	model_rank_ids=[0, 1],
	rnn_type='lstm',
)

In [ ]:
# callbacks
reducer_params = dict(
    monitor='val_rmse',
    factor=0.05,
    patience=5,
    verbose=3,
    mode='min',
    min_delta=5e-7,
    cooldown=5,
    min_lr=1e-6,
)

lr_reducer = tf.keras.callbacks.ReduceLROnPlateau(**reducer_params)

In [ ]:
lstm_retrainer.start(callbacks=[lr_reducer])

In [ ]:
lstm_histories = lstm_retrainer.get_model_histories
lstm_histories

# **Evaluation**

In [ ]:
lstm_retrainer.is_retrain = True
lstm_retrainer.is_retrain

In [ ]:
lstm_retrainer.evaluate(
	split_mode='train',
	model_version='latest'
)